In [111]:
import pandas as pd
import numpy as np
from datasets import load_dataset, DatasetDict,Dataset

In [163]:
data = pd.read_csv("top1000_travel_subreddit.csv",index_col="Unnamed: 0")

In [164]:
def first_and_last_quarter(data):
    if not data:
        return [], []  # Return empty lists if the input list is empty

    part_length = int(len(data) * .25)
    first_quarter = data[:part_length]
    last_quarter = data[-part_length:] if part_length > 0 else []
    
    return first_quarter, last_quarter

In [165]:
data.head()

,title,selftext,creation_date,id,url,upvote_ratio,ups,downs,score,link_flair_css_class,comments
0,Passport Questions & Issues Megathread (2023),NOTE: October 2023 **If the US Government has ...,2023-01-01 12:56:19,100t75r,https://www.reddit.com/r/travel/comments/100t7...,0.99,544,0,544,question,"['As expected based on the last few years, the..."
1,"U.S. Department of State - ""Worldwide Caution""",U.S. Department of State issued a new travel a...,2023-10-19 10:41:36,17bouw5,https://www.reddit.com/r/travel/comments/17bou...,0.94,742,0,742,advice,['Yes. They are routinely issued when there is...
2,Why isn’t Heathrow widely flagged as a nightma...,The whole experience at Heathrow made me decid...,2023-11-09 15:36:20,17rqf0d,https://www.reddit.com/r/travel/comments/17rqf...,0.84,192,0,192,question,"[""It's talked about a fair bit 😅\n\nLHR is the..."
3,Was refused boarding to a AirAsia flight to Vi...,Hello a few hours have passed since I have fig...,2023-11-09 08:17:48,17rgkfl,https://www.reddit.com/r/travel/comments/17rgk...,0.86,197,0,197,question,['Where did you buy these flights? This sounds...
4,Air France Rant,I have to fly Air France several times a year ...,2023-11-09 13:20:25,17rncol,https://www.reddit.com/r/travel/comments/17rnc...,0.83,67,0,67,NaN,['why not report it to the EU aviation authori...


In [166]:
for i,r in data.iterrows():
    q = r['selftext']
    split = r['comments'].split('\\n\\n')
    first_q, last_q = first_and_last_quarter(split)
    chosen_txt = ' '.join(first_q).replace('[','').replace(']','')
    rejected_txt = ' '.join(last_q).replace('[','').replace(']','')
    data.at[i,'chosen'] = chosen_txt
    data.at[i,'rejected'] = rejected_txt

In [152]:
for i,r in data.iterrows():
    q = r['selftext']
    split = r['comments'].split('\\n\\n')
    first_q, last_q = first_and_last_quarter(split)
    chosen_txt = ' '.join(first_q).replace('[','').replace(']','')
    rejected_txt = ' '.join(last_q).replace('[','').replace(']','')
    data.at[i,'chosen'] = [{ "content": q, "role": "user" },{ "content": chosen_txt, "role": "assistant" } ]
    data.at[i,'rejected'] = [{ "content": q, "role": "user" },{ "content": rejected_txt, "role": "assistant" } ]



In [167]:
processed_df = data.replace('',np.nan).dropna().reset_index(drop=True)
ds = processed_df[['selftext','chosen','rejected']]

In [168]:
ds.to_csv("rm_data_no_format.csv")

In [169]:
dataset = Dataset.from_pandas(ds)

In [170]:
dataset.push_to_hub(repo_id="chriztopherton/wanderchat-rm-no-format")

Uploading the dataset shards: 100%|██████████| 1/1 [00:00<00:00,  1.06it/s]


CommitInfo(commit_url='https://huggingface.co/datasets/chriztopherton/wanderchat-rm-no-format/commit/77af48a991bc280d172c1177af7511d13a124f4e', commit_message='Upload dataset', commit_description='', oid='77af48a991bc280d172c1177af7511d13a124f4e', pr_url=None, pr_revision=None, pr_num=None)

In [171]:
dataset[0]

{'selftext': 'NOTE: October 2023 **If the US Government has a Shutdown Passport Processing will slow down. There will not be a way around this. Do not trust 3rd party services.**\n\n___\n\n\n**For the last few years we\'ve been getting multiple posts a day on American Passport Questions. To help alleviate the amount of posts, and make sure everyone is getting the same information I\'ve created this Megathread.**\n\n___\n\nTHE BEST ADVICE IS: \n\n* APPLY AND RENEW EARLY. \n* ALWAYS CHECK YOUR PASSPORT\'S EXPIRATION DATE. \n* PUT YOUR PASSPORT IN A SAFE AND SECURE PLACE WHERE YOU WILL NOT LOSE IT. \n* DO NOT WAIT TO CHECK EXPIRATION OR LOCATION OF PASSPORT THE NIGHT BEFORE. CHECK IT EARLY. \n\n\n___\n\n**[The 2022 Thread Can Be Found Here.](https://www.reddit.com/r/travel/comments/v2m73y/passport_questions_issues_megathread_2022/)**\n\n**[The Winter 2021 / Spring 2022 Passport Megathread can be found here](https://www.reddit.com/r/travel/comments/ps3cvv/us_passport_questions_issues_megat